# Integrate datasets

In [1]:
import pandas as pd
import os
from IPython.display import clear_output
import requests
import json
from datetime import datetime

## Load prepared SDG dataset¶

In [2]:
# Load the Excel file
xls = pd.ExcelFile('../inputs/sdg_dataset.xlsx')

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Read each sheet into a separate DataFrame and store them in a dictionary
dfs_sdg = {}
for sheet in sheet_names:
    dfs_sdg[sheet.lower()] = pd.read_excel(xls, sheet_name=sheet).fillna('')


In [3]:
dfs_sdg.keys()

dict_keys(['data', 'schema', 'cl_geo'])

In [4]:
# Load the Excel file
xls = pd.ExcelFile('../inputs/ilo_dataset.xlsx')

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Read each sheet into a separate DataFrame and store them in a dictionary
dfs_ilo = {}
for sheet in sheet_names:
    dfs_ilo[sheet.lower()] = pd.read_excel(xls, sheet_name=sheet).fillna('')


In [5]:
dfs_ilo.keys()

dict_keys(['data', 'schema', 'cl_geo'])

In [6]:

# Load the Excel file
xls = pd.ExcelFile('../inputs/Example_mappings.xlsx')

# Get the names of all sheets in the Excel file
sheet_names = xls.sheet_names

# Read each sheet into a separate DataFrame and store them in a dictionary
dfs_mappings = {}
for sheet in sheet_names:
    dfs_mappings[sheet.lower()] = pd.read_excel(xls, sheet_name=sheet).fillna('')


In [7]:
dfs_mappings.keys()

dict_keys(['concept_mapping__undata_ilo', 'concept_mapping__undata_sdg', 'enum_mapping_geo__undata_sdg', 'enum_mapping_geo__undata_ilo'])

## Utility functions

In [8]:
def collapse_columns(df):
    """
    Collapse repeated columns into a single column with lists of strings.
    Leave single-occurrence columns as they are.
    """
    collapsed_df = pd.DataFrame()
    for col in set(df.columns):
        if sum(df.columns == col) > 1:
            collapsed_series = df.loc[:, col].apply(lambda row: [str(i) for i in row], axis=1)
            collapsed_df[col] = collapsed_series.astype(str)
        else:
            collapsed_df[col] = df[col]
    return collapsed_df

## Integrate datasets into a single table

In [9]:
dfs_sdg['schema'].head(3)

,ConceptID,ConceptName,Coded,Role
0,sdg:SERIES_CODE,Series Code,True,Dimension
1,sdg:SERIES_DESCRIPTION,Series Description,False,Attribute
2,sdg:VARIABLE_CODE,Variable Code,True,Attribute


In [10]:
dfs_ilo['schema'].head(3)

,ConceptID,ConceptName,Coded,Role
0,ilo:DATAFLOW,Dataflow,False,Attribute
1,ilo:REF_AREA,Ref Area,True,Dimension
2,ilo:FREQ,Freq,True,Dimension


In [11]:
dfs_mappings['concept_mapping__undata_ilo'].head(10)

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_date,author_id,subject_source,subject_source_version,object_source,object_source_version,confidence
0,undata:ACTIVE_DIMS,Active dimensions,,,,,,,,,,,
1,undata:BASE_PERIOD,Base period,,,,,,,,,,,
2,undata:DATAFLOW,Dataflow,skos:exactMatch,ilo:DATAFLOW,Dataflow,,,,,,,,
3,undata:DECIMALS,Decimals,skos:exactMatch,ilo:DECIMALS,Decimals,,,,,,,,
4,undata:FOOTNOTE,Footnote,skos:exactMatch,ilo:NOTE_CLASSIF,Note Classif,,,,,,,,
5,undata:FOOTNOTE,Footnote,skos:exactMatch,ilo:NOTE_INDICATOR,Note Indicator,,,,,,,,
6,undata:FOOTNOTE,Footnote,skos:exactMatch,ilo:NOTE_SOURCE,Note Source,,,,,,,,
7,undata:FREQ,Frequency,skos:exactMatch,ilo:FREQ,Freq,,,,,,,,
8,undata:GEOGRAPHY,Geography,skos:exactMatch,ilo:REF_AREA,Ref Area,,,,,,,,
9,undata:GEOGRAPHY_TYPE,Geography type,,,,,,,,,,,


## 1a. Change column names

In [12]:
list(dfs_sdg['data'])

['sdg:SERIES_CODE',
 'sdg:SERIES_DESCRIPTION',
 'sdg:VARIABLE_CODE',
 'sdg:VARIABLE_DESCRIPTION',
 'sdg:VARIABLE_ACTIVE_DIMS',
 'sdg:GEOGRAPHY_CODE',
 'sdg:GEOGRAPHY_NAME',
 'sdg:GEOGRAPHY_TYPE',
 'sdg:GEO_AREA_CODE',
 'sdg:GEO_AREA_NAME',
 'sdg:CITIES',
 'sdg:SAMPLING_STATIONS',
 'sdg:IS_LATEST_PERIOD',
 'sdg:TIME_PERIOD',
 'sdg:TIME_DETAIL',
 'sdg:TIME_COVERAGE',
 'sdg:FREQ',
 'sdg:AGE',
 'sdg:SEX',
 'sdg:OBS_VALUE',
 'sdg:VALUE_TYPE',
 'sdg:UPPER_BOUND',
 'sdg:LOWER_BOUND',
 'sdg:UNIT_MEASURE',
 'sdg:UNIT_MULT',
 'sdg:BASE_PERIOD',
 'sdg:NATURE',
 'sdg:SOURCE',
 'sdg:GEO_INFO_URL',
 'sdg:FOOT_NOTE',
 'sdg:REPORTING_TYPE',
 'sdg:OBS_STATUS',
 'sdg:RELEASE_STATUS',
 'sdg:RELEASE_NAME']

In [13]:
# Create a dictionary from 'old' to 'new' columns
m = dfs_mappings['concept_mapping__undata_sdg'].loc[(dfs_mappings['concept_mapping__undata_sdg']['subject_id'] != '') & (dfs_mappings['concept_mapping__undata_sdg']['object_id'] != '')]
rename_dict = dict(zip(m['object_id'], m['subject_id']))
# Rename columns in DataFrame X
dfs_sdg['data'].rename(columns=rename_dict, inplace=True)
dfs_sdg['data'].head(6)
list(dfs_sdg['data'])

['undata:MEASURE',
 'sdg:SERIES_DESCRIPTION',
 'undata:VARIABLE',
 'sdg:VARIABLE_DESCRIPTION',
 'undata:ACTIVE_DIMS',
 'undata:GEOGRAPHY',
 'sdg:GEOGRAPHY_NAME',
 'undata:GEOGRAPHY_TYPE',
 'sdg:GEO_AREA_CODE',
 'sdg:GEO_AREA_NAME',
 'sdg:CITIES',
 'sdg:SAMPLING_STATIONS',
 'undata:IS_LATEST_PERIOD',
 'undata:TIME_PERIOD',
 'undata:TIME_DETAIL',
 'undata:TIME_COVERAGE',
 'undata:FREQ',
 'undata:MEASURE',
 'undata:MEASURE',
 'undata:OBS_VALUE',
 'undata:VALUE_TYPE',
 'undata:UPPER_BOUND',
 'undata:LOWER_BOUND',
 'undata:UNIT_MEASURE',
 'undata:UNIT_MULT',
 'undata:BASE_PERIOD',
 'undata:NATURE',
 'undata:SOURCE',
 'sdg:GEO_INFO_URL',
 'undata:FOOTNOTE',
 'undata:REPORTING_TYPE',
 'undata:OBS_STATUS',
 'undata:RELEASE_STATUS',
 'undata:RELEASE']

In [14]:
#rename_dict

## 2a. Transcode values for geography concept

In [15]:
dfs_mappings['enum_mapping_geo__undata_sdg'].head(4)

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_date,author_id,subject_source,subject_source_version,object_source,object_source_version,confidence
0,undata:GEOGRAPHY/G00000010,Abu Dhabi,,,,,NaT,,,,,,
1,undata:GEOGRAPHY/G00000020,Afghanistan,skos:exactMatch,sdg:GEOGRAPHY_CODE/4,Afghanistan,semapv:LexicalSimilarityThresholdMatching,NaT,,,,,,1.0
2,undata:GEOGRAPHY/G00000030,Ajman,,,,,NaT,,,,,,
3,undata:GEOGRAPHY/G00000040,Åland Islands,skos:exactMatch,sdg:GEOGRAPHY_CODE/248,Åland Islands,semapv:LexicalSimilarityThresholdMatching,NaT,,,,,,1.0


In [16]:
# Create a dictionary from 'old' to 'new' columns
transcode_dict = dict(zip(dfs_mappings['enum_mapping_geo__undata_sdg']['object_id'], dfs_mappings['enum_mapping_geo__undata_sdg']['subject_id']))
dfs_sdg['data']['undata:GEOGRAPHY'] = dfs_sdg['data']['undata:GEOGRAPHY'].replace(transcode_dict)


## Collapse repeated columns into single column

In [17]:
dfs_sdg['data'] = collapse_columns(dfs_sdg['data'])

In [18]:
dfs_sdg['data']

,undata:SOURCE,undata:UNIT_MEASURE,sdg:GEO_AREA_CODE,undata:RELEASE_STATUS,undata:VALUE_TYPE,undata:REPORTING_TYPE,undata:FOOTNOTE,sdg:VARIABLE_DESCRIPTION,undata:FREQ,undata:RELEASE,...,sdg:CITIES,undata:TIME_PERIOD,undata:OBS_STATUS,undata:GEOGRAPHY_TYPE,undata:UNIT_MULT,sdg:GEOGRAPHY_NAME,undata:VARIABLE,sdg:SAMPLING_STATIONS,sdg:GEO_INFO_URL,undata:NATURE
0,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2000,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
1,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2001,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
2,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2002,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
3,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2003,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
4,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2004,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29275,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/894,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2015,sdg:OBS_STATUS/E,Country,,Zambia,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y_GE25__SEX--M,,,sdg:NATURE/M
29276,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/894,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2016,sdg:OBS_STATUS/E,Country,,Zambia,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y_GE25__SEX--M,,,sdg:NATURE/M
29277,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/894,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2017,sdg:OBS_STATUS/E,Country,,Zambia,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y_GE25__SEX--M,,,sdg:NATURE/M
29278,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/894,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2018,sdg:OBS_STATUS/E,Country,,Zambia,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y_GE25__SEX--M,,,sdg:NATURE/M


## 1b. Change column names

In [19]:
list(dfs_ilo['data'])

['ilo:DATAFLOW',
 'ilo:REF_AREA',
 'ilo:FREQ',
 'ilo:MEASURE',
 'ilo:QTL',
 'ilo:TIME_PERIOD',
 'ilo:OBS_VALUE',
 'ilo:OBS_STATUS',
 'ilo:UNIT_MEASURE_TYPE',
 'ilo:UNIT_MEASURE',
 'ilo:UNIT_MULT',
 'ilo:SOURCE',
 'ilo:NOTE_SOURCE',
 'ilo:NOTE_INDICATOR',
 'ilo:NOTE_CLASSIF',
 'ilo:DECIMALS',
 'ilo:UPPER_BOUND',
 'ilo:LOWER_BOUND']

In [20]:
m = dfs_mappings['concept_mapping__undata_ilo'].loc[(dfs_mappings['concept_mapping__undata_ilo']['subject_id'] != '') & (dfs_mappings['concept_mapping__undata_sdg']['object_id'] != '')]
rename_dict = dict(zip(m['object_id'], m['subject_id']))

# Rename columns in DataFrame X
dfs_ilo['data'].rename(columns=rename_dict, inplace=True)
dfs_ilo['data'].head(6)

,ilo:DATAFLOW,undata:GEOGRAPHY,undata:FREQ,undata:MEASURE,undata:MEASURE,undata:TIME_PERIOD,undata:OBS_VALUE,undata:OBS_STATUS,undata:UNIT_MEASURE_TYPE,ilo:UNIT_MEASURE,undata:UNIT_MULT,undata:SOURCE,undata:FOOTNOTE,undata:FOOTNOTE,undata:FOOTNOTE,ilo:DECIMALS,undata:UPPER_BOUND,undata:LOWER_BOUND
0,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_01,2013,0.44,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
1,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_02,2013,1.01,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
2,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_03,2013,1.70,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
3,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_04,2013,2.88,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
4,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_05,2013,4.79,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
5,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:REF_AREA/AFG,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_06,2013,6.89,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,


## 2a. Transcode values for geography concept

In [21]:
dfs_mappings['enum_mapping_geo__undata_ilo'].head(4)

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_date,author_id,subject_source,subject_source_version,object_source,object_source_version,confidence
0,undata:GEOGRAPHY/G00000010,Abu Dhabi,,,,,,,,,,,
1,undata:GEOGRAPHY/G00000020,Afghanistan,skos:exactMatch,ilo:REF_AREA/AFG,Afghanistan,,,,,,,,
2,undata:GEOGRAPHY/G00000030,Ajman,,,,,,,,,,,
3,undata:GEOGRAPHY/G00000040,Åland Islands,,,,,,,,,,,


In [22]:
# Create a dictionary from 'old' to 'new' columns
transcode_dict = dict(zip(dfs_mappings['enum_mapping_geo__undata_ilo']['object_id'], dfs_mappings['enum_mapping_geo__undata_ilo']['subject_id']))
dfs_ilo['data']['undata:GEOGRAPHY'] = dfs_sdg['data']['undata:GEOGRAPHY'].replace(transcode_dict)
dfs_ilo['data']

,ilo:DATAFLOW,undata:GEOGRAPHY,undata:FREQ,undata:MEASURE,undata:MEASURE,undata:TIME_PERIOD,undata:OBS_VALUE,undata:OBS_STATUS,undata:UNIT_MEASURE_TYPE,ilo:UNIT_MEASURE,undata:UNIT_MULT,undata:SOURCE,undata:FOOTNOTE,undata:FOOTNOTE,undata:FOOTNOTE,ilo:DECIMALS,undata:UPPER_BOUND,undata:LOWER_BOUND
0,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_01,2013,0.44,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_02,2013,1.01,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
2,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_03,2013,1.70,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
3,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_04,2013,2.88,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
4,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_05,2013,4.79,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21675,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_06,2020,1.40,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
21676,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_07,2020,2.47,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
21677,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_08,2020,5.45,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,
21678,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,ilo:FREQ/A,ilo:MEASURE/LAP_2LID_RT,ilo:QTL/QTL_DECILE_09,2020,17.10,ilo:OBS_STATUS/M,ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT,ilo:UNIT_MULT/0,ILO - Modelled Estimates,,,,1,,


## Collapse repeated columns into single column

In [23]:
dfs_ilo['data'] = collapse_columns(dfs_ilo['data'])

In [24]:
dfs_ilo['data']

,undata:FOOTNOTE,undata:FREQ,undata:SOURCE,undata:MEASURE,ilo:DECIMALS,ilo:DATAFLOW,undata:GEOGRAPHY,undata:TIME_PERIOD,undata:OBS_VALUE,undata:UNIT_MEASURE_TYPE,undata:OBS_STATUS,undata:UPPER_BOUND,undata:LOWER_BOUND,undata:UNIT_MULT,ilo:UNIT_MEASURE
0,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,0.44,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
1,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,1.01,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
2,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,1.70,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
3,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,2.88,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
4,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,4.79,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21675,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,2020,1.40,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
21676,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,2020,2.47,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
21677,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,2020,5.45,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
21678,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00800370,2020,17.10,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT


---
# Create integrated dataset

In [25]:
x1 = dfs_sdg['data'].copy(deep=True)
x1 = x1.reset_index(drop=True)
x1.head(3)

,undata:SOURCE,undata:UNIT_MEASURE,sdg:GEO_AREA_CODE,undata:RELEASE_STATUS,undata:VALUE_TYPE,undata:REPORTING_TYPE,undata:FOOTNOTE,sdg:VARIABLE_DESCRIPTION,undata:FREQ,undata:RELEASE,...,sdg:CITIES,undata:TIME_PERIOD,undata:OBS_STATUS,undata:GEOGRAPHY_TYPE,undata:UNIT_MULT,sdg:GEOGRAPHY_NAME,undata:VARIABLE,sdg:SAMPLING_STATIONS,sdg:GEO_INFO_URL,undata:NATURE
0,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2000,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
1,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2001,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N
2,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,2002,sdg:OBS_STATUS/E,Region,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N


In [26]:
x2 = dfs_ilo['data'].copy(deep=True)
x2 = x2.reset_index(drop=True)
x2.head(3)

,undata:FOOTNOTE,undata:FREQ,undata:SOURCE,undata:MEASURE,ilo:DECIMALS,ilo:DATAFLOW,undata:GEOGRAPHY,undata:TIME_PERIOD,undata:OBS_VALUE,undata:UNIT_MEASURE_TYPE,undata:OBS_STATUS,undata:UPPER_BOUND,undata:LOWER_BOUND,undata:UNIT_MULT,ilo:UNIT_MEASURE
0,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,0.44,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
1,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,1.01,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT
2,"['', '', '']",ilo:FREQ/A,ILO - Modelled Estimates,"['ilo:MEASURE/LAP_2LID_RT', 'ilo:QTL/QTL_DECIL...",1,ILO:DF_LAP_2LID_QTL_RT(1.0),undata:GEOGRAPHY/G00100000,2013,1.70,ilo:UNIT_MEASURE_TYPE/RT,ilo:OBS_STATUS/M,,,ilo:UNIT_MULT/0,ilo:UNIT_MEASURE/PT


In [27]:
result = pd.concat([x1, x2], axis=0)


In [28]:
result

,undata:SOURCE,undata:UNIT_MEASURE,sdg:GEO_AREA_CODE,undata:RELEASE_STATUS,undata:VALUE_TYPE,undata:REPORTING_TYPE,undata:FOOTNOTE,sdg:VARIABLE_DESCRIPTION,undata:FREQ,undata:RELEASE,...,undata:UNIT_MULT,sdg:GEOGRAPHY_NAME,undata:VARIABLE,sdg:SAMPLING_STATIONS,sdg:GEO_INFO_URL,undata:NATURE,ilo:DECIMALS,ilo:DATAFLOW,undata:UNIT_MEASURE_TYPE,ilo:UNIT_MEASURE
0,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N,NaN,NaN,NaN,NaN
1,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N,NaN,NaN,NaN,NaN
2,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N,NaN,NaN,NaN,NaN
3,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N,NaN,NaN,NaN,NaN
4,"Source = ILO modelled estimates, Nov. 2022",sdg:UNIT_MEASURE/PT,sdg:GEO_AREA_CODE/1,Published,sdg:VALUE_TYPE/Float,sdg:REPORTING_TYPE/G,,Employed population below international povert...,,2023.Q2.G.01,...,,World,sdg:VARIABLE_CODE/SI_POV_EMP1@AGE--Y15T24,,,sdg:NATURE/N,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21675,ILO - Modelled Estimates,NaN,NaN,NaN,NaN,NaN,"['', '', '']",NaN,ilo:FREQ/A,NaN,...,ilo:UNIT_MULT/0,NaN,NaN,NaN,NaN,NaN,1.0,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT
21676,ILO - Modelled Estimates,NaN,NaN,NaN,NaN,NaN,"['', '', '']",NaN,ilo:FREQ/A,NaN,...,ilo:UNIT_MULT/0,NaN,NaN,NaN,NaN,NaN,1.0,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT
21677,ILO - Modelled Estimates,NaN,NaN,NaN,NaN,NaN,"['', '', '']",NaN,ilo:FREQ/A,NaN,...,ilo:UNIT_MULT/0,NaN,NaN,NaN,NaN,NaN,1.0,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT
21678,ILO - Modelled Estimates,NaN,NaN,NaN,NaN,NaN,"['', '', '']",NaN,ilo:FREQ/A,NaN,...,ilo:UNIT_MULT/0,NaN,NaN,NaN,NaN,NaN,1.0,ILO:DF_LAP_2LID_QTL_RT(1.0),ilo:UNIT_MEASURE_TYPE/RT,ilo:UNIT_MEASURE/PT


In [30]:
result.to_excel("../inputs/integrated_dataset2.xlsx", index=False)

In [31]:
result_integrated = result[[col for col in result.columns if col.startswith('undata:')]]
columns_integrated = list(result_integrated)
columns_integrated

['undata:SOURCE',
 'undata:UNIT_MEASURE',
 'undata:RELEASE_STATUS',
 'undata:VALUE_TYPE',
 'undata:REPORTING_TYPE',
 'undata:FOOTNOTE',
 'undata:FREQ',
 'undata:RELEASE',
 'undata:TIME_DETAIL',
 'undata:MEASURE',
 'undata:GEOGRAPHY',
 'undata:OBS_VALUE',
 'undata:UPPER_BOUND',
 'undata:ACTIVE_DIMS',
 'undata:LOWER_BOUND',
 'undata:BASE_PERIOD',
 'undata:IS_LATEST_PERIOD',
 'undata:TIME_COVERAGE',
 'undata:TIME_PERIOD',
 'undata:OBS_STATUS',
 'undata:GEOGRAPHY_TYPE',
 'undata:UNIT_MULT',
 'undata:VARIABLE',
 'undata:NATURE',
 'undata:UNIT_MEASURE_TYPE']

In [32]:
value_cols = [x for x in columns_integrated if x not in ['undata:GEOGRAPHY', 'undata:TIME_PERIOD', 
                                                         'undata:MEASURE',
                                                         'undata:GEOGRAPHY_TYPE', 
                                                         'undata:TIME_DETAIL', 'undata:TIME_COVERAGE',
                                                         'undata:IS_LATEST_PERIOD']]

In [33]:
pivoted_df = pd.pivot_table(result_integrated, 
                            values = value_cols,
                            index=['undata:GEOGRAPHY', 'undata:TIME_PERIOD'],
                            columns=['undata:MEASURE'],
                            aggfunc='first') 
pivoted_df

undata:ACTIVE_DIMS  \
undata:MEASURE                                ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/F']   
undata:GEOGRAPHY           undata:TIME_PERIOD                                                                  
undata:GEOGRAPHY/G00000050 2000                                                   ['AGE', 'SEX']               
                           2001                                                   ['AGE', 'SEX']               
                           2002                                                   ['AGE', 'SEX']               
                           2003                                                   ['AGE', 'SEX']               
                           2004                                                   ['AGE', 'SEX']               
...                                                                                          ...               
undata:GEOGRAPHY/G00800500 2018                                                   ['AGE', 'SEX']               
                           2019                                                   ['AGE', 'SEX']               
                           2020                                                              NaN               
                           2021                                                              NaN               
                           2022                                                              NaN               

                                                                                                              \
undata:MEASURE                                ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/M']   
undata:GEOGRAPHY           undata:TIME_PERIOD                                                                  
undata:GEOGRAPHY/G00000050 2000                                                   ['AGE', 'SEX']               
                           2001                                                   ['AGE', 'SEX']               
                           2002                                                   ['AGE', 'SEX']               
                           2003                                                   ['AGE', 'SEX']               
                           2004                                                   ['AGE', 'SEX']               
...                                                                                          ...               
undata:GEOGRAPHY/G00800500 2018                                                   ['AGE', 'SEX']               
                           2019                                                   ['AGE', 'SEX']               
                           2020                                                              NaN               
                           2021                                                              NaN               
                           2022                                                              NaN               

                                                                                                               \
undata:MEASURE                                ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/_T']   
undata:GEOGRAPHY           undata:TIME_PERIOD                                                                   
undata:GEOGRAPHY/G00000050 2000                                                          ['AGE']                
                           2001                                                          ['AGE']                
                           2002                                                          ['AGE']                
                           2003                                                          ['AGE']                
                           2004                                                          ['AGE']                
...                                             

In [35]:
pivoted_df.reset_index(inplace=True)
pivoted_df

undata:GEOGRAPHY undata:TIME_PERIOD  \
undata:MEASURE                                                  
0               undata:GEOGRAPHY/G00000050               2000   
1               undata:GEOGRAPHY/G00000050               2001   
2               undata:GEOGRAPHY/G00000050               2002   
3               undata:GEOGRAPHY/G00000050               2003   
4               undata:GEOGRAPHY/G00000050               2004   
...                                    ...                ...   
3675            undata:GEOGRAPHY/G00800500               2018   
3676            undata:GEOGRAPHY/G00800500               2019   
3677            undata:GEOGRAPHY/G00800500               2020   
3678            undata:GEOGRAPHY/G00800500               2021   
3679            undata:GEOGRAPHY/G00800500               2022   

                                                           undata:ACTIVE_DIMS  \
undata:MEASURE ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/F']   
0                                                  ['AGE', 'SEX']               
1                                                  ['AGE', 'SEX']               
2                                                  ['AGE', 'SEX']               
3                                                  ['AGE', 'SEX']               
4                                                  ['AGE', 'SEX']               
...                                                           ...               
3675                                               ['AGE', 'SEX']               
3676                                               ['AGE', 'SEX']               
3677                                                          NaN               
3678                                                          NaN               
3679                                                          NaN               

                                                                               \
undata:MEASURE ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/M']   
0                                                  ['AGE', 'SEX']               
1                                                  ['AGE', 'SEX']               
2                                                  ['AGE', 'SEX']               
3                                                  ['AGE', 'SEX']               
4                                                  ['AGE', 'SEX']               
...                                                           ...               
3675                                               ['AGE', 'SEX']               
3676                                               ['AGE', 'SEX']               
3677                                                          NaN               
3678                                                          NaN               
3679                                                          NaN               

                                                                                \
undata:MEASURE ['sdg:SERIES_CODE/SI_POV_EMP1', 'sdg:AGE/Y15T24', 'sdg:SEX/_T']   
0                                                         ['AGE']                
1                                                         ['AGE']                
2                                                         ['AGE']                
3                                                         ['AGE']                
4                                                         ['AGE']                
...                                                           ...                
3675                                                      ['AGE']                
3676                                                      ['AGE']                
3677                                                          NaN                
3678                                                          NaN                
3679                                                          NaN                

     

In [38]:
pivoted_df.columns = ['_'.join(col) for col in pivoted_df.columns]

with pd.ExcelWriter("../inputs/integrated_dataset3.xlsx", engine='xlsxwriter') as writer:
    pivoted_df.to_excel(writer, index=False, sheet_name='Sheet1')
    